# Lab 3 - Module 0: When Straight Lines Fail

**Learning Objectives:**
- Recognize patterns that cannot be separated by straight lines
- Understand why linear models have fundamental limitations
- Build motivation for nonlinear methods

**Time:** ~10 minutes

---

**Remember from Lab 1:** You learned how to fit lines to data by adjusting slope and intercept.

**Remember from Lab 2:** Gradient descent can automatically find the best parameters.

**Today's Question:** What if NO straight line works at all?

## 1. Setup: Create Three Datasets

We'll create three simple classification problems:
- **Dataset 1:** Two clouds of points (separable)
- **Dataset 2:** XOR pattern - four corners (not separable)
- **Dataset 3:** Circular ring - inner vs outer (not separable)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import FloatSlider, interact, VBox, HTML
from IPython.display import display

# Set random seed for reproducibility
np.random.seed(42)

# Dataset 1: Linearly separable (two Gaussian blobs)
n_points = 100
X1_class0 = np.random.randn(n_points // 2, 2) * 0.5 + np.array([-1.5, -1.5])
X1_class1 = np.random.randn(n_points // 2, 2) * 0.5 + np.array([1.5, 1.5])
X1 = np.vstack([X1_class0, X1_class1])
y1 = np.hstack([np.zeros(n_points // 2), np.ones(n_points // 2)])

# Dataset 2: XOR (four corners)
n_per_corner = 25
X2_corners = np.array([
    [-1.5, -1.5], [1.5, -1.5],  # Class 0 (bottom corners)
    [-1.5, 1.5], [1.5, 1.5]      # Class 1 (top corners)
])
X2 = []
y2 = []
for i, corner in enumerate(X2_corners):
    points = np.random.randn(n_per_corner, 2) * 0.3 + corner
    X2.append(points)
    # XOR pattern: corners 0,3 are class 0, corners 1,2 are class 1
    label = 0 if i % 3 == 0 else 1
    y2.append(np.ones(n_per_corner) * label)
X2 = np.vstack(X2)
y2 = np.hstack(y2)

# Dataset 3: Concentric circles
n_per_circle = 50
# Inner circle (class 0)
angles_inner = np.random.uniform(0, 2*np.pi, n_per_circle)
radius_inner = np.random.uniform(0.3, 0.8, n_per_circle)
X3_inner = np.column_stack([
    radius_inner * np.cos(angles_inner),
    radius_inner * np.sin(angles_inner)
])
# Outer ring (class 1)
angles_outer = np.random.uniform(0, 2*np.pi, n_per_circle)
radius_outer = np.random.uniform(1.5, 2.2, n_per_circle)
X3_outer = np.column_stack([
    radius_outer * np.cos(angles_outer),
    radius_outer * np.sin(angles_outer)
])
X3 = np.vstack([X3_inner, X3_outer])
y3 = np.hstack([np.zeros(n_per_circle), np.ones(n_per_circle)])

print("✓ Three datasets created!")
print("\nDataset 1: Two clouds of points")
print("Dataset 2: XOR pattern (four corners)")
print("Dataset 3: Circular ring (inner vs outer)")

## 2. Visual Preview of All Three Datasets

Before you try to separate them with lines, let's see what they look like.

In [ ]:
# Preview all three datasets
fig, axes = plt.subplots(1, 3, figsize=(15, 4), dpi=100)

datasets = [(X1, y1, "Dataset 1: Two Clouds"),
            (X2, y2, "Dataset 2: XOR Pattern"),
            (X3, y3, "Dataset 3: Circular Ring")]

for ax, (X, y, title) in zip(axes, datasets):
    # Plot class 0 (blue) and class 1 (red)
    ax.scatter(X[y == 0, 0], X[y == 0, 1], c='blue', s=50, alpha=0.6, label='Class 0 (Blue)', edgecolors='k')
    ax.scatter(X[y == 1, 0], X[y == 1, 1], c='red', s=50, alpha=0.6, label='Class 1 (Red)', edgecolors='k')
    ax.set_xlabel('x₁', fontsize=11)
    ax.set_ylabel('x₂', fontsize=11)
    ax.set_title(title, fontsize=12, fontweight='bold')
    ax.legend(fontsize=9)
    ax.grid(True, alpha=0.3)
    ax.set_aspect('equal')

plt.tight_layout()
plt.show()

print("\nYour challenge: Can you draw a STRAIGHT LINE to separate blue from red in each dataset?")
print("Try it in the interactive tool below!")

## 3. Interactive: Try to Separate with a Straight Line

**Your task:** Adjust the slope and intercept sliders to try to draw a line that separates:
- **Blue points** on one side
- **Red points** on the other side

**The line equation:** `x₂ = slope × x₁ + intercept`

Points **above** the line are classified as one class, points **below** are the other class.

In [ ]:
def plot_dataset_with_line(dataset_num, slope, intercept):
    """
    Plot the selected dataset with a separating line.
    
    Args:
        dataset_num: 1, 2, or 3
        slope: slope of the line
        intercept: y-intercept of the line
    """
    # Select dataset
    if dataset_num == 1:
        X, y, title = X1, y1, "Dataset 1: Two Clouds"
    elif dataset_num == 2:
        X, y, title = X2, y2, "Dataset 2: XOR Pattern"
    else:
        X, y, title = X3, y3, "Dataset 3: Circular Ring"
    
    # Create figure
    fig, ax = plt.subplots(figsize=(8, 8), dpi=100)
    
    # Plot data points
    ax.scatter(X[y == 0, 0], X[y == 0, 1], c='blue', s=80, alpha=0.6, 
              label='Class 0 (Blue)', edgecolors='k', linewidths=1.5)
    ax.scatter(X[y == 1, 0], X[y == 1, 1], c='red', s=80, alpha=0.6, 
              label='Class 1 (Red)', edgecolors='k', linewidths=1.5)
    
    # Plot the line: x₂ = slope * x₁ + intercept
    x_range = np.linspace(X[:, 0].min() - 1, X[:, 0].max() + 1, 100)
    y_line = slope * x_range + intercept
    ax.plot(x_range, y_line, 'g-', linewidth=3, label=f'Line: x₂ = {slope:.2f}×x₁ + {intercept:.2f}')
    
    # Compute accuracy (simple metric: how many are correctly classified)
    # Points above line (x₂ > slope*x₁ + intercept) → predicted class 1
    # Points below line → predicted class 0
    predicted = (X[:, 1] > slope * X[:, 0] + intercept).astype(int)
    accuracy = np.mean(predicted == y) * 100
    
    # Formatting
    ax.set_xlabel('x₁', fontsize=13)
    ax.set_ylabel('x₂', fontsize=13)
    ax.set_title(f'{title}\nAccuracy: {accuracy:.1f}%', fontsize=14, fontweight='bold')
    ax.legend(fontsize=11, loc='best')
    ax.grid(True, alpha=0.3)
    ax.set_aspect('equal')
    ax.set_xlim(X[:, 0].min() - 0.5, X[:, 0].max() + 0.5)
    ax.set_ylim(X[:, 1].min() - 0.5, X[:, 1].max() + 0.5)
    
    plt.tight_layout()
    plt.show()
    
    # Feedback
    if accuracy > 95:
        print(f"🎉 Excellent! {accuracy:.1f}% accuracy - this line separates the classes well!")
    elif accuracy > 80:
        print(f"👍 Good! {accuracy:.1f}% accuracy - pretty close, but not perfect.")
    elif accuracy > 60:
        print(f"😐 Okay. {accuracy:.1f}% accuracy - this line does better than random guessing.")
    else:
        print(f"😕 Hmm. {accuracy:.1f}% accuracy - this line doesn't separate well.")
        if accuracy < 50:
            print("   (You might be doing worse than flipping a coin!)")

# Create interactive widget
print("Interactive Line Separator")
print("="*70)
print("Adjust the sliders to try to separate blue from red points.")
print("The line equation is: x₂ = slope × x₁ + intercept\n")

interact(
    plot_dataset_with_line,
    dataset_num=widgets.IntSlider(min=1, max=3, step=1, value=1, description='Dataset:'),
    slope=FloatSlider(min=-3, max=3, step=0.1, value=1.0, description='Slope:'),
    intercept=FloatSlider(min=-3, max=3, step=0.1, value=0.0, description='Intercept:')
);

## 4. The Key Discovery

After trying all three datasets, you should have discovered:

### Dataset 1 (Two Clouds): ✅ **Solvable**
You can find a line that separates blue from red nearly perfectly!

### Dataset 2 (XOR Pattern): ❌ **Impossible**
No matter what line you draw:
- If the line separates bottom-left from top-right, it fails on the other two corners
- You'd need **TWO lines** to separate this pattern
- Best you can do is ~50% accuracy (random guessing!)

### Dataset 3 (Circular Ring): ❌ **Impossible**
No straight line can separate an inner circle from an outer ring:
- Any line will have both blue and red points on both sides
- You'd need a **CIRCLE** (curved boundary) to separate this
- Best you can do is ~50-60% accuracy

---

**The problem:** Linear models (straight lines) **fundamentally cannot solve** some patterns.

**The solution (coming in this lab):** We need models that can create **curved** decision boundaries!

## 5. Visualize the Fundamental Limitation

Let's see what the "best possible" straight lines look like for each dataset.

In [ ]:
# Best manually-tuned lines for each dataset
best_lines = [
    (1.0, 0.0, "Works!"),     # Dataset 1: slope=1, intercept=0
    (0.0, 0.0, "Fails!"),     # Dataset 2: ANY line fails
    (0.0, 0.0, "Fails!")      # Dataset 3: ANY line fails
]

fig, axes = plt.subplots(1, 3, figsize=(16, 5), dpi=100)
datasets = [(X1, y1, "Dataset 1: Two Clouds"),
            (X2, y2, "Dataset 2: XOR Pattern"),
            (X3, y3, "Dataset 3: Circular Ring")]

for ax, (X, y, title), (slope, intercept, verdict) in zip(axes, datasets, best_lines):
    # Plot data
    ax.scatter(X[y == 0, 0], X[y == 0, 1], c='blue', s=60, alpha=0.6, 
              label='Class 0', edgecolors='k', linewidths=1)
    ax.scatter(X[y == 1, 0], X[y == 1, 1], c='red', s=60, alpha=0.6, 
              label='Class 1', edgecolors='k', linewidths=1)
    
    # Plot line
    x_range = np.linspace(X[:, 0].min() - 1, X[:, 0].max() + 1, 100)
    y_line = slope * x_range + intercept
    ax.plot(x_range, y_line, 'g-', linewidth=3, label='Best line')
    
    # Compute accuracy
    predicted = (X[:, 1] > slope * X[:, 0] + intercept).astype(int)
    accuracy = np.mean(predicted == y) * 100
    
    # Formatting
    ax.set_xlabel('x₁', fontsize=11)
    ax.set_ylabel('x₂', fontsize=11)
    ax.set_title(f'{title}\n{verdict} (Accuracy: {accuracy:.1f}%)', 
                fontsize=11, fontweight='bold')
    ax.legend(fontsize=9)
    ax.grid(True, alpha=0.3)
    ax.set_aspect('equal')

plt.tight_layout()
plt.show()

print("\nConclusion:")
print("="*70)
print("✅ Dataset 1: Linear models work great!")
print("❌ Dataset 2 (XOR): Linear models fail completely")
print("❌ Dataset 3 (Circles): Linear models fail completely")
print("\n💡 We need something more powerful than straight lines!")
print("   → That's what activation functions and perceptrons provide!")

## Questions for Your Answer Sheet

**Q1.** Which dataset(s) can be separated by a straight line?

**Q2.** For the XOR pattern (Dataset 2), what happens no matter how you adjust the line? Why can't ANY straight line separate it?

**Q3.** Why can't a straight line separate the circular ring pattern (Dataset 3)? What kind of boundary shape would you need instead?

## Next Steps

1. **Answer Q1, Q2, Q3** on your answer sheet
2. **Return to the LMS** and continue to Module 1
3. In Module 1, you'll see how "bending space" can help solve these impossible problems!